In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
azure_oai_key = os.getenv("AZURE_OAI_KEY")
azure_oai_deployment = os.getenv("AZURE_OAI_DEPLOYMENT")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_KEY")
search_index = os.getenv("AZURE_SEARCH_INDEX")


In [2]:
print(azure_oai_endpoint) # make sure this is gpt-5-turbo, other models will run out of token quota

https://oai-o1-preview-apil-instance.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview


In [3]:
print(search_index)
print(azure_oai_deployment)

brochures-index
gpt-35-turbo


## Define a data source configuration
A separate ada-002 vectorizing model is used to consume a blob storage and push the data into a Azure AI Search index (named 'brochure-index').

In [14]:
extension_config = dict(data_sources = [
    {
        "type": "AzureCognitiveSearch",
        "parameters": {
            "endpoint": search_endpoint,
            "key": search_key,
            "indexName": search_index
        }
    }])
                        

In [5]:
from openai import AzureOpenAI

In [6]:

client = AzureOpenAI(
    azure_endpoint=azure_oai_endpoint,
    api_key=azure_oai_key,
    api_version="2023-09-01-preview")

In [7]:
input_text = "Tell me something about Dubai"
messages = [
    {"role": "system", "content": "You are a helpful travel agent"},
    {"role": "user", "content": input_text}
]

In [13]:
response = client.chat.completions.create(
    model=azure_oai_deployment,
    messages=messages,
    temperature=0.5,
    max_tokens=1000,
    extra_body= {
        "data_sources": [
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "key": search_key,
                        "indexName": search_index
                    }
                }
        ]
    }
)
print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': 'Unrecognized request argument supplied: data_sources', 'type': 'invalid_request_error', 'param': None, 'code': None}}